In [2]:
import os

In [3]:
s3_keys_path = '~/.keys/lsst_keys.json'
swift_keys_path = '~/.keys/lsst-swift-credentials.json'
creds = {}

#for swift
with open(os.path.expanduser(swift_keys_path), 'r') as swiftkeys:
    for line in swiftkeys.readlines():
        if 'user' in line:
            creds['ST_USER'] = line.split('"')[3]
        elif 'secret_key' in line:
            creds['ST_KEY'] = line.split('"')[3]
creds['ST_AUTH'] = 'https://s3.echo.stfc.ac.uk/auth/1.0'

#Some Swift operations expect environment variables
#Here we just mirror the creds dict to the os.environ dict
for k, v in creds.items():
    os.environ[k] = v

In [4]:
import bucket_manager.bucket_manager as bm

## Swift Connection

In [5]:
connection = bm.get_conn_swift()

In [6]:
# %%timeit
containers = [ container['name'] for container in connection.get_account()[1] ]

In [7]:
for i, container in enumerate(containers):
    print(i, container)

0 DRP
1 LSST-IR-FUSION
2 LSST-IR-FUSION-Butlers
3 LSST-IR-FUSION-TEST
4 LSST-IR-FUSION-rdsip005
5 LSST-IR-FUSION-test-zip-processing
6 LSST-IR-FUSION-testfromopenstack
7 LSST-IR-FUSION_gen3_conversion
8 dmu4
9 lsst-dac
10 lsst-drp-config
11 lsst-test


In [22]:
container = containers[2]
print(container)

LSST-IR-FUSION-Butlers


In [23]:
import numpy as np

In [ ]:
# %%timeit
with open('temp_filelist.csv', 'w') as f:
    f.write('name\n')
    num_obj = 0
    for _ in connection.get_container(container, full_listing=True)[1]:
        num_obj+=1
    print(num_obj)
    for i in np.arange(0, num_obj, 100000):
        print(i)
        for obj in connection.get_container(container, full_listing=True, limit=100000, marker=i)[1]:
            f.write(obj['name'] + '\n')

KeyboardInterrupt: 

2025-03-10 16:47:31,532 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-03-10 16:47:31,532 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-03-10 16:47:31,532 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-03-10 16:47:31,532 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/distributed/nanny.py", line 1021, in _run
    asyncio_run(run(), loop_factory=get_loop_factory())
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/distributed/nanny.py", line 1021, in _run
    asyncio_run(run(), loop_factory=get_loop_factory())
  File "/home/dave/miniconda3/envs/lsst

In [10]:
from dask import dataframe as ddf
import distributed
client = distributed.Client()
client

/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32959 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:32959/status,
Dashboard: http://127.0.0.1:32959/status,Workers: 4
Total threads: 12,Total memory: 7.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41597,Workers: 4
Dashboard: http://127.0.0.1:32959/status,Total threads: 12
Started: Just now,Total memory: 7.43 GiB
Comm: tcp://127.0.0.1:33317,Total threads: 3
Dashboard: http://127.0.0.1:35111/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:46123,


In [11]:
!wc -l temp_filelist.csv

13 temp_filelist.csv


In [12]:
objects_swift_dask = ddf.read_csv('temp_filelist.csv', blocksize=2)

In [13]:
objects_swift_dask

,name
npartitions=186,
,string
,...
...,...
,...
,...


In [14]:
def delete_obj_swift(obj):
    try:
        connection.delete_object(container, obj)
    except Exception as e:
        return e
    return True

In [15]:
import pandas as pd

In [16]:
objects_swift_dask['deleted'] = objects_swift_dask['name'].apply(delete_obj_swift, meta=pd.Series(dtype=bool))

In [17]:
objects_swift_dask

,name,deleted
npartitions=186,,
,string,bool
,...,...
...,...,...
,...,...
,...,...


In [18]:
# %%timeit
objects_swift = objects_swift_dask.compute()

In [19]:
objects_swift

,name,deleted
0,dummy/collated_4.zip.metadata,True
0,dummy/collated_5.zip.metadata,True
0,dummy/collated_55.zip.metadata,True
0,dummy/collated_59.zip.metadata,True
0,dummy/collated_6.zip.metadata,True
0,dummy/collated_61.zip.metadata,True
0,dummy/collated_62.zip.metadata,True
0,dummy/collated_63.zip.metadata,True
0,dummy/collated_64.zip.metadata,True
0,dummy/collated_65.zip.metadata,True
